## Mask R-CNN - Inspect Trained Model

Code and visualizations to test, debug, and evaluate the Mask R-CNN model.

In [1]:
from datetime import date, datetime, timezone, timedelta

exp_day = str(date.today())

KST = timezone(timedelta(hours=9))
time_record = datetime.now(KST)
_day = str(time_record)[:10]
_time = str(time_record.time())[:8]

print(datetime.now(KST))

2022-03-12 15:59:13.992706+09:00


In [2]:
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import keras

# Root directory of the project
ROOT_DIR = os.path.abspath("../..")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
import visualize
from mrcnn.model import log

%matplotlib inline 

import total_seg
import total2_bbox

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

SEG_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_seg.h5")  # epoch 145
BBOX_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_bbox.h5")  # epoch 100

/home/s/anaconda3/envs/maskrcnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/s/anaconda3/envs/maskrcnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/s/anaconda3/envs/maskrcnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/s/anaconda3/envs/maskrcnn/lib/python3.7/site-pack

## Configurations

In [3]:
class SInferenceConfig(total_seg.ParkConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
class BInferenceConfig(total2_bbox.ParkConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1


seg_config = SInferenceConfig()
seg_config.display()

bbox_config = BInferenceConfig()
bbox_config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Notebook Preferences

In [4]:
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
TEST_MODE = "inference"

In [5]:
def get_ax(rows=1, cols=1, size=16):
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# import glob
# tet = glob.glob('/data/DatasetSeg_Final/*')
# print(tet)

## Load Validation Dataset

In [ ]:
bbox_data_path = 'Dataset2d'

# BBOX 모델 Load Validation Dataset
bbox_dataset = total2_bbox.ParkDataset()
bbox_dataset.load_park(bbox_data_path, "test")
bbox_dataset.prepare()
print("bbox_dataset Images: {}\nClasses: {}".format(len(bbox_dataset.image_ids), bbox_dataset.class_names))

In [ ]:
seg_data_path = 'Dataset2dSeg'

# SEG 모델 Load Validation Dataset
seg_dataset = total_seg.ParkDataset()
seg_dataset.load_park(seg_data_path, "test")
seg_dataset.prepare()
print("seg_dataset Images: {}\nClasses: {}".format(len(seg_dataset.image_ids), seg_dataset.class_names))

Dataset2dSeg-test/라벨링데이터
Dataset2dSeg-test/원천데이터


## Load Model

In [ ]:
seg_model = modellib.MaskRCNN(mode="inference", model_dir=SEG_MODEL_PATH, config=seg_config)
bbox_model = modellib.MaskRCNN(mode="inference", model_dir=BBOX_MODEL_PATH, config=bbox_config)

print("Loading seg_weights ", SEG_MODEL_PATH)
seg_model.load_weights(SEG_MODEL_PATH, by_name=True)

print("Loading bbox_weights ", BBOX_MODEL_PATH)
bbox_model.load_weights(BBOX_MODEL_PATH, by_name=True)

In [ ]:
seg_class_names = ["BG", "Parking Space", "Driveable Space"]

bbox_class_names = ["BG", "Car", "Van", "Other Vehicle", "Motorbike", "Bicycle", "Electric Scooter", "Adult", "Child", "Stroller", "Shopping Cart", "Gate Arm", 
               "Parking Block", "Speed Bump", "Traffic Pole", "Traffic Cone", "Traffic Drum", "Traffic Barricade", "Cylindrical Bollard", "U-shaped Bollard", 
               "Other Road Barriers", "No Parking Stand", "Adjustable Parking Pole", "Waste Tire", "Planter Barrier", "Water Container", "Movable Obstacle", 
               "Barrier Gate", "Electric Car Charger", "Parking Meter", "Parking Sign", "Traffic Light", "Pedestrian Light", "Street Sign", "Disabled Parking Space", 
               "Pregnant Parking Space", "Electric Car Parking Space", "Two-wheeled Vehicle Parking Space", "Other Parking Space"]

## Run Detection(SEG)

In [ ]:
# SEG 모델
import random
import visualize
image_id = random.choice(seg_dataset.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(seg_dataset, seg_config, image_id, use_mini_mask=False)
info = seg_dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       seg_dataset.image_reference(image_id)))

results = seg_model.detect([image], verbose=1)

# 결과 보기
ax = get_ax(1)
r = results[0]
visualize.display_instances_seg(image, r['rois'], r['masks'], r['class_ids'], 
                            seg_class_names, r['scores'], ax=ax,
                            title="Predictions")
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

### Precision-Recall(SEG)

In [ ]:
def d_compute_matches_seg(gt_boxes, gt_class_ids, gt_masks,
                    pred_boxes, pred_class_ids, pred_scores, pred_masks,
                    iou_threshold=0.5, score_threshold=0.0):
    """Finds matches between prediction and ground truth instances.

    Returns:
        gt_match: 1-D array. For each GT box it has the index of the matched
                  predicted box.
        pred_match: 1-D array. For each predicted box, it has the index of
                    the matched ground truth box.
        overlaps: [pred_boxes, gt_boxes] IoU overlaps.
    """
    # Trim zero padding
    # TODO: cleaner to do zero unpadding upstream
    gt_boxes = utils.trim_zeros(gt_boxes)
    gt_masks = gt_masks[..., :gt_boxes.shape[0]]
    pred_boxes = utils.trim_zeros(pred_boxes)
    pred_scores = pred_scores[:pred_boxes.shape[0]]
    # Sort predictions by score from high to low
    indices = np.argsort(pred_scores)[::-1]
    pred_boxes = pred_boxes[indices]
    pred_class_ids = pred_class_ids[indices]
    pred_scores = pred_scores[indices]
    pred_masks = pred_masks[..., indices]

    # Compute IoU overlaps [pred_masks, gt_masks]
    overlaps = utils.compute_overlaps_masks(pred_masks,gt_masks)

    # Loop through predictions and find matching ground truth boxes
    match_count = 0
    pred_match = -1 * np.ones([pred_boxes.shape[0]])
    gt_match = -1 * np.ones([gt_boxes.shape[0]])

    # print("(compute_matches) overlaps : ",overlaps)

    # print("(compute_matches) pred_class_ids : ",pred_class_ids)
    # print("(compute_matches) gt_class_ids : ",gt_class_ids)

    comp = []

    for i in range(len(pred_boxes)):
        # Find best matching ground truth box
        # 1. Sort matches by score
        sorted_ixs = np.argsort(overlaps[i])[::-1]
        # print("(compute_matches) sorted_ixs : ",sorted_ixs)
        # 2. Remove low scores
        low_score_idx = np.where(overlaps[i, sorted_ixs] < score_threshold)[0]
        if low_score_idx.size > 0:
            sorted_ixs = sorted_ixs[:low_score_idx[0]]
        # 3. Find the match
        for j in sorted_ixs:
            # If ground truth box is already matched, go to next one
            if gt_match[j] > -1:
                continue
            # If we reach IoU smaller than the threshold, end the loop
            iou = overlaps[i, j]
            # print('(compute_matches) **gt_class_ids1 : ',gt_class_ids[j])
            # print('(compute_matches) **pred_class_ids1 : ',pred_class_ids[i])
            # print('(compute_matches) **pred_scores : ',pred_scores[i])
            # print('(compute_matches) **iou : ',iou)

            if iou < iou_threshold:
                break

            # print('(compute_matches) **gt_class_ids2 : ',gt_class_ids[j])
            # print('(compute_matches) **pred_class_ids2 : ',pred_class_ids[i])
            # print('(compute_matches) **iou : ',iou)

            comp.append([gt_class_ids[j],pred_class_ids[i],iou,pred_scores[i]])
            
            # Do we have a match?
            if pred_class_ids[i] == gt_class_ids[j]:
                match_count += 1
                gt_match[j] = i
                pred_match[i] = j
                break
    # print('(compute_matches) gt_match2',gt_match)
    # print('(compute_matches) pred_match2',pred_match)

    for i,gt_m in enumerate(gt_match) :
        if gt_m == -1 :
            # print('(compute_matches) **gt_class_ids2 : ',gt_class_ids[i])
            comp.append([gt_class_ids[i]])


    return gt_match, pred_match, overlaps,comp


def d_compute_ap_seg(gt_boxes, gt_class_ids, gt_masks,
               pred_boxes, pred_class_ids, pred_scores, pred_masks,
               iou_threshold=0.5):
    """Compute Average Precision at a set IoU threshold (default 0.5).

    Returns:
    mAP: Mean Average Precision
    precisions: List of precisions at different class score thresholds.
    recalls: List of recall values at different class score thresholds.
    overlaps: [pred_boxes, gt_boxes] IoU overlaps.
    """
    # Get matches and overlaps
    gt_match, pred_match, overlaps,comp = d_compute_matches_seg(
        gt_boxes, gt_class_ids, gt_masks,
        pred_boxes, pred_class_ids, pred_scores, pred_masks,
        iou_threshold)

    # Compute precision and recall at each prediction box step
    precisions = np.cumsum(pred_match > -1) / (np.arange(len(pred_match)) + 1)
    recalls = np.cumsum(pred_match > -1).astype(np.float32) / len(gt_match)

    # Pad with start and end values to simplify the math
    precisions = np.concatenate([[0], precisions, [0]])
    recalls = np.concatenate([[0], recalls, [1]])

    # Ensure precision values decrease but don't increase. This way, the
    # precision value at each recall threshold is the maximum it can be
    # for all following recall thresholds, as specified by the VOC paper.
    for i in range(len(precisions) - 2, -1, -1):
        precisions[i] = np.maximum(precisions[i], precisions[i + 1])

    # Compute mean AP over recall range
    indices = np.where(recalls[:-1] != recalls[1:])[0] + 1
    mAP = np.sum((recalls[indices] - recalls[indices - 1]) *
                 precisions[indices])

    return mAP, precisions, recalls, overlaps,comp


In [ ]:
AP, precisions, recalls, overlaps,comp = d_compute_ap_seg(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])
visualize.plot_precision_recall(AP, precisions, recalls)

In [ ]:
# Grid of ground truth objects and their predictions
visualize.plot_overlaps(gt_class_id, r['class_ids'], r['scores'],
                        overlaps, seg_class_names)

## Run Detection(BBOX)

In [ ]:
# BBOX 모델
import random
image_id = random.choice(bbox_dataset.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(bbox_dataset, bbox_config, image_id, use_mini_mask=False)
info = bbox_dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       bbox_dataset.image_reference(image_id)))
# Run object detection
results = bbox_model.detect([image], verbose=1)

# Display results
ax = get_ax(1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            bbox_class_names, r['scores'], ax=ax,
                            title="Predictions")
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

### Precision-Recall(BBOX)

In [ ]:
def d_compute_matches(gt_boxes, gt_class_ids, gt_masks,
                    pred_boxes, pred_class_ids, pred_scores, pred_masks,
                    iou_threshold=0.1, score_threshold=0.0):
    """Finds matches between prediction and ground truth instances.

    Returns:
        gt_match: 1-D array. For each GT box it has the index of the matched
                  predicted box.
        pred_match: 1-D array. For each predicted box, it has the index of
                    the matched ground truth box.
        overlaps: [pred_boxes, gt_boxes] IoU overlaps.
    """
    # Trim zero padding
    # TODO: cleaner to do zero unpadding upstream
    gt_boxes = utils.trim_zeros(gt_boxes)
    gt_masks = gt_masks[..., :gt_boxes.shape[0]]
    pred_boxes = utils.trim_zeros(pred_boxes)
    pred_scores = pred_scores[:pred_boxes.shape[0]]
    # Sort predictions by score from high to low
    indices = np.argsort(pred_scores)[::-1]
    pred_boxes = pred_boxes[indices]
    pred_class_ids = pred_class_ids[indices]
    pred_scores = pred_scores[indices]
    pred_masks = pred_masks[..., indices]

    # Compute IoU overlaps [pred_masks, gt_masks]
    overlaps = utils.compute_overlaps(pred_boxes,gt_boxes)

    # Loop through predictions and find matching ground truth boxes
    match_count = 0
    pred_match = -1 * np.ones([pred_boxes.shape[0]])
    gt_match = -1 * np.ones([gt_boxes.shape[0]])

    comp = []

    for i in range(len(pred_boxes)):
        # Find best matching ground truth box
        # 1. Sort matches by score
        sorted_ixs = np.argsort(overlaps[i])[::-1]
        # print("(compute_matches) sorted_ixs : ",sorted_ixs)
        # 2. Remove low scores
        low_score_idx = np.where(overlaps[i, sorted_ixs] < score_threshold)[0]
        if low_score_idx.size > 0:
            sorted_ixs = sorted_ixs[:low_score_idx[0]]
        # 3. Find the match
        for j in sorted_ixs:
            # If ground truth box is already matched, go to next one
            if gt_match[j] > -1:
                continue
            iou = overlaps[i, j]

            if iou < iou_threshold:
                break

            comp.append([gt_class_ids[j],pred_class_ids[i],iou,pred_scores[i]])
            
            # Do we have a match?
            if pred_class_ids[i] == gt_class_ids[j]:
                match_count += 1
                gt_match[j] = i
                pred_match[i] = j
                break
    # print('(compute_matches) gt_match2',gt_match)
    # print('(compute_matches) pred_match2',pred_match)

    for i,gt_m in enumerate(gt_match) :
        if gt_m == -1 :
            # print('(compute_matches) **gt_class_ids2 : ',gt_class_ids[i])
            comp.append([gt_class_ids[i]])


    return gt_match, pred_match, overlaps,comp


def d_compute_ap(gt_boxes, gt_class_ids, gt_masks,
               pred_boxes, pred_class_ids, pred_scores, pred_masks,
               iou_threshold=0.1):
    """Compute Average Precision at a set IoU threshold (default 0.5).

    Returns:
    mAP: Mean Average Precision
    precisions: List of precisions at different class score thresholds.
    recalls: List of recall values at different class score thresholds.
    overlaps: [pred_boxes, gt_boxes] IoU overlaps.
    """
    # Get matches and overlaps
    gt_match, pred_match, overlaps,comp = d_compute_matches(
        gt_boxes, gt_class_ids, gt_masks,
        pred_boxes, pred_class_ids, pred_scores, pred_masks,
        iou_threshold)

    # Compute precision and recall at each prediction box step
    precisions = np.cumsum(pred_match > -1) / (np.arange(len(pred_match)) + 1)
    recalls = np.cumsum(pred_match > -1).astype(np.float32) / len(gt_match)

    # Pad with start and end values to simplify the math
    precisions = np.concatenate([[0], precisions, [0]])
    recalls = np.concatenate([[0], recalls, [1]])

    # Ensure precision values decrease but don't increase. This way, the
    # precision value at each recall threshold is the maximum it can be
    # for all following recall thresholds, as specified by the VOC paper.
    for i in range(len(precisions) - 2, -1, -1):
        precisions[i] = np.maximum(precisions[i], precisions[i + 1])

    # Compute mean AP over recall range
    indices = np.where(recalls[:-1] != recalls[1:])[0] + 1
    mAP = np.sum((recalls[indices] - recalls[indices - 1]) *
                 precisions[indices])

    return mAP, precisions, recalls, overlaps,comp


In [ ]:
# Draw precision-recall curve

AP, precisions, recalls, overlaps,comp = d_compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])
visualize.plot_precision_recall(AP, precisions, recalls)

In [ ]:
# Grid of ground truth objects and their predictions

visualize.plot_overlaps(gt_class_id, r['class_ids'], r['scores'],
                        overlaps, bbox_class_names)

### Compute mAP @ IoU=50 on Batch of Images

In [ ]:
name_dict_bbox ={
                1 : "Car" ,
                2 : "Van" ,
                3 : "Other Vehicle" , 
                4 : "Motorbike" ,
                5 : "Bicycle",
                6 : "Electric Scooter",
                7 : "Adult",
                8 : "Child",
                9 : "Stroller",
                10 : "Shopping Cart" ,
                11 : "Gate Arm",
                12 : "Parking Block",
                13 : "Speed Bump",
                14 : "Traffic Pole",
                15 : "Traffic Cone",
                16 : "Traffic Drum",
                17 : "Traffic Barricade",
                18 : "Cylindrical Bollard",
                19 : "U-shaped Bollard",
                20 : "Other Road Barriers",
                21 : "No Parking Stand", 
                22 : "Adjustable Parking Pole",
                23 : "Waste Tire",
                24 : "Planter Barrier",
                25 : "Water Container",
                26 : "Movable Obstacle",
                27 : "Barrier Gate",
                28 : "Electric Car Charger",
                29 : "Parking Meter",
                30 : "Parking Sign",
                31 : "Traffic Light",
                32 : "Pedestrian Light",
                33 : "Street Sign",
                34 : "Disabled Parking Space",
                35 : "Pregnant Parking Space",
                36 : "Electric Car Parking Space",
                37 : "Two-wheeled Vehicle Parking Space",
                38 : "Other Parking Space" ,
                }
name_dict_seg ={
                1 : "Parking Space",
                2 : "Driveable Space"
                }

In [ ]:
#SEG모델
import csv

image_ids = seg_dataset.image_ids
APs = []

with open('listiou_seg.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    
    for image_id in image_ids:
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(seg_dataset, seg_config,
                                   image_id, use_mini_mask=False)
        molded_images = np.expand_dims(modellib.mold_image(image, seg_config), 0)
        results = seg_model.detect([image], verbose=0)
        r = results[0]
        data_name = [seg_dataset.image_reference(image_id)]

        AP, precisions, recalls, overlaps,comp =\
            d_compute_ap_seg(gt_bbox, gt_class_id, gt_mask,
                             r["rois"], r["class_ids"], r["scores"], r['masks'])
        for c in comp :
            c[0] = name_dict_seg[c[0]]
            if len(c) >2:
              c[1] = name_dict_seg[c[1]]
            comp_n = data_name + c
            writer.writerow(comp_n) 
        APs.append(AP)
    
print("mAP: ", np.mean(APs))

In [ ]:
#BBOX모델
import csv

image_ids = bbox_dataset.image_ids
APs = []

with open('listiou_bbox.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    
    for image_id in image_ids:
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(bbox_dataset, bbox_config,
                                   image_id, use_mini_mask=False)
        molded_images = np.expand_dims(modellib.mold_image(image, bbox_config), 0)
        results = bbox_model.detect([image], verbose=0)
        r = results[0]
        data_name = [bbox_dataset.image_reference(image_id)]

        AP, precisions, recalls, overlaps,comp =\
            d_compute_ap(gt_bbox, gt_class_id, gt_mask,
                             r["rois"], r["class_ids"], r["scores"], r['masks'])
        for c in comp :
            c[0] = name_dict_bbox[c[0]]
            if len(c) >2:
              c[1] = name_dict_bbox[c[1]]
            comp_n = data_name + c
            writer.writerow(comp_n) 
        APs.append(AP)
print("mAP @ IoU=50: ", np.mean(APs))